# Importações

In [ ]:
# Biblioteca utilizada para realizar operações matriciais
import numpy as np
# Utilizada para inicializar os pesos e biases com números aleatórios
import random
# Utilizada para plotar as imagens
import matplotlib.pyplot as plt

# Definição da rede neural

## funções úteis

In [ ]:
# Função sigmoid
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
# Derivada da função sigmoid
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
"""
    @description: Recebe uma imagem (vetor) como parâmetro e realiza a impressão
    (plotagem) na tela
    @params: Um vetor contendo os dados da imagem 28x28
    @return: void
"""
def plot_image(image):
    image = image.reshape((28, 28))

    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

## definição do modelo

In [ ]:
class Network(object):
    """
        @description: A lista 'sizes' possui o número de neurônios na camada
        respectiva. Por exemplo, se a lista for [2, 3, 1], então será uma rede
        com 3 camadas, com a primeira contendo 2 neurônios, a segunda 3 e a
        terceira apenas 1. Os biases e pesos da rede são iniciados aleatoriamente,
        usando a distribuição gaussiana com média 0 e variância 1. Observe que a
        primeira camada é uma camada de entrada e, por convenção, não atribuimos
        biases a ela, considerando que os biases são usados apenas para computar
        a saída das outras camadas à frente.
        @params: Um N-array contendo, em cada posição (layer), o número de
        neurônios daquela camada.
        @return: void
    """
    def __init__(self, sizes) -> None:
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
    
    """
        @description: O array 'image' representa a imagem do número a ser predito.
        Esta função realiza a predição de qual algarismo representa esta imagem e
        retorna o valor da probalidade.
        @params: Um N-array (srqt(N) = dim da imagem) representando os pixels da
        imagem.
        @return: Um inteiro de 0-9 representando o valor predito e um número real
        entre 0-1 representando a probalidade.
    """
    def __call__(self, image):
        pred = self.feedforward(image)

        greater = pred[0]
        greater_index = -1
        index = 0

        for x in pred:
            if (x > greater):
                greater = x
                greater_index = index
            
            index += 1
        
        return greater_index

    """
        @description: Retorna a saída da rede neural se 'a' é uma entrada
        @params: um N-array representando uma entrada (N é a dimensão da entrada).
        @return: um N-array representando as probabilidades de cada neurônio
        da camada de saída (N é a dimensão da saída, isto é, a quantidade de 
        neurônios da última camada).
    """
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    """
        @description: Treina a rede neural usando o gradiente estocástico mini-batch.
        O 'training_data' é um N-array de tuplas '(x, y)' representando a entrada
        de treino e a saída desejada. Se 'test_data' é fornecido, então a rede
        será testada com 'test_data' a cada epoch de treino e o progresso parcial
        será impresso. Isso é muito útil para acompanhar o progresso, mas retardará
        substancialmente o processo.
        @params:
            training_data: um N-array contendo, em cada posição, um tupla '(x, y)'
            onde x é um M-array representando a imagem para treino e y a saída
            desejada.
            epochs: um inteiro que representa a quantidades de epochs que o algoritmo
            deve executar.
            mini_batch_size: um inteiro que representa o tamanho do mini_batch.
            eta: um número real que representa a taxa de aprendizado.
            test_data (opcional): um N-array contendo, em cada posição, um tupla 
            '(x, y)' onde x é um M-array representando a imagem para teste e y
            a saída desejada.
        @return: void
    """
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        training_data = list(training_data)

        if (test_data): test_data = list(test_data)

        if (test_data): n_teste = len(test_data)

        n = len(training_data)

        for j in range(epochs):
            random.shuffle(training_data)

            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)
            ]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)

            if (test_data):
                print("Epoch {0}: {1}/  {2}".format(j, self.evaluate(test_data), n_teste))
            else:
                print("Epoch {0} complete".format(j))

    """
        @description: Atualiza os pesos e biases da rede aplicando o gradiente
        descendente usando backpropagation para cada mini-batch. O 'mini-batch'
        é um N-array de tuplas '(x, y)' e 'eta' é a taxa de aprendizado.
        @params:
            mini_batch: um N-array de tuplas '(x, y)'.
            eta: um número real que representa a taxa de aprendizado
        @return:
    """  
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
    
    """
        @description: Retorna uma tupla '(nabla_b, nabla_w)' que representa o
        gradiente para a função de custo C_x. 'nabla_b' e 'nabla_w' são, camada
        por camada, listas de Numpy arrays, similar a self.biases e self.weights.
        @params:
            x: um N-array que representa uma imagem da entrada.
            y: um inteiro que representa o valor de saída desejado.
        @return: uma tupla '(nabla_b, nabla_w)'.
    """
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activations = [x] 
        zs = [] 
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    """
        @description: Retorna o número de entradas de teste que a rede neural
        retornou uma saída correta. Observe que uma saída correta da rede neural
        representa o índice de qualquer neurônio com maior ativação na camada
        de saída.
        @params: test_data -> um N-array de tuplas '(x, y)' onde x representa um
        M-array da imagem e y representa o valor de saída desejado.
        @return: Um inteiro que representa a soma de todas imagens que foram
        preditas corretamente.
    """
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    """
        @description: Retorna o vetor de derivadas parciais C_x / a das funções
        de ativação de saída.
        @params:
            output_activations: Um número real representando a ativação da camada
            de saída
            y: um inteiro representando o valor desejado
        @return: Um número real representando o custo
    """
    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

# Utilização

## importações

In [ ]:
import mnist_loader

## Treinando o modelo

Instanciando o modelo

In [ ]:
model = Network([784, 30, 10])

Importando os dados para treino

In [ ]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

Executando o gradiente descendente estocástico

In [ ]:
model.SGD(training_data, 30, 10, 3, test_data)

## Validando o modelo

In [ ]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [ ]:
first_image_vd = list(validation_data)[1200][0]

In [ ]:
plot_image(first_image_vd)

In [ ]:
model(first_image_vd)